In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import json
from datetime import datetime

import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')


# this enables relative path imports
import os
from dotenv import load_dotenv
load_dotenv()
_PROJECT_PATH: str = os.environ["_project_path"]
_PICKLED_DATA_FILENAME: str = os.environ["_pickled_data_filename"]

import sys
from pathlib import Path
project_path = Path(_PROJECT_PATH)
sys.path.append(str(project_path))

import config_v2 as cfg

from library_report_v2 import Cleaning as cln
from library_report_v2 import Graphing as grp
from library_report_v2 import Processing as pro
from library_report_v2 import Configuration as repcfg

Baseline en config_v2.py: ['2024-01-01', '2024-04-29']
Study en config_v2.py: ['2024-04-29', '2024-05-06']


In [2]:
def show_response_contents(df):
    print("The response contains:")
    print(json.dumps(list(df['variable'].unique()), sort_keys=True, indent=4))
    print(json.dumps(list(df['device'].unique()), sort_keys=True, indent=4))

In [3]:
df = pd.read_pickle(project_path / 'data' / _PICKLED_DATA_FILENAME)
show_response_contents(df)

The response contains:
[
    "ilu-consumo-activa",
    "front-potencia-activa",
    "tr",
    "consumo-energia-reactiva-capacitiva",
    "kw-tr",
    "consumo-energia-reactiva-inductiva",
    "factor-de-potencia",
    "ilu-potencia-activa",
    "front-consumo-activa",
    "consumo-energia-reactiva-total",
    "aa-consumo-activa",
    "area",
    "aa-potencia-activa"
]
[
    "bc57",
    "bc10",
    "bc35",
    "bc8",
    "bc36",
    "bc01-iluminacion-girardot",
    "bc71",
    "bc---s.a.-valledupar",
    "bc---floridablanca",
    "bc14",
    "bc56",
    "bc18",
    "bc60",
    "bc30",
    "bc45",
    "bc21",
    "bc2",
    "bc12",
    "bc16",
    "bc44",
    "bc-santa-monica",
    "bc40",
    "bc15",
    "bc71-2",
    "bc61",
    "bc33",
    "bc42",
    "bc41",
    "bc-291-las-palmas",
    "bc11",
    "bc-megamall",
    "gateway-1-bc-uc",
    "bc65",
    "bc26",
    "bc32",
    "bc38",
    "bc43",
    "bc31",
    "bc58t",
    "bc---s.a.-granada",
    "bc49",
    "bc27",
    "bc64",
    

In [4]:
df = df.sort_values(by=['variable','datetime'])
df = pro.datetime_attributes(df)

df_bl, df_st = pro.split_into_baseline_and_study(df, baseline=cfg.BASELINE, study=cfg.STUDY, inclusive='left')
df_bl['Periodo'] = 'Baseline'
df_st['Periodo'] = 'Estudio'

past_w = df_bl.loc[cfg.PAST_WEEK[0]:cfg.PAST_WEEK[1]]

In [5]:
df_pa = df.query("variable == 'front-potencia-activa'").copy()
front = df.query("variable == 'front-consumo-activa'").copy()

In [ ]:
front_hour = front.groupby(by=["variable"]).resample('1h').sum().round(2).reset_index().set_index('datetime')
front_hour = pro.datetime_attributes(front_hour)

In [6]:
front

,value,variable,device,device_name,hour,day,dow,cont_dow,week,month,year
datetime,,,,,,,,,,,
2023-01-01 00:00:00-05:00,7.08,front-consumo-activa,bc---s.a.-valledupar,BC 524 - S.A. Valledupar,0,1,domingo,6.0,52,1,2023
2023-01-01 00:00:00-05:00,37.19,front-consumo-activa,bc---s.a.-granada,BC 821 GRANADA CALI,0,1,domingo,6.0,52,1,2023
2023-01-01 00:00:00-05:00,3.41,front-consumo-activa,bc-816-1---calle-10,BC 816 - Calle 10,0,1,domingo,6.0,52,1,2023
2023-01-01 00:00:00-05:00,2.72,front-consumo-activa,bc71-2,BC 793 - Profesionales,0,1,domingo,6.0,52,1,2023
2023-01-01 00:00:00-05:00,6.88,front-consumo-activa,bc71,BC 787 - Bocagrande Carrera Tercera,0,1,domingo,6.0,52,1,2023
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-06 00:00:00-05:00,4.90,front-consumo-activa,bc7,BC 253 - Puerta del Norte,0,6,lunes,0.0,19,5,2024
2024-05-06 00:00:00-05:00,3.00,front-consumo-activa,bc6,BC 197 - Guatapuri,0,6,lunes,0.0,19,5,2024
2024-05-06 00:00:00-05:00,4.05,front-consumo-activa,bc4,BC 78 - El Cacique,0,6,lunes,0.0,19,5,2024


In [7]:
front_hour = front.groupby(by=["variable", "device_name"]).resample('1h').mean().round(2).reset_index().set_index('datetime')
front_hour = pro.datetime_attributes(front_hour)

In [19]:
front_hour

,device_name,hour,value,day,dow,cont_dow,week,year
0,BC 044 - Piedecuesta,0,3.08,1,domingo,6.00,52,2023
1,BC 044 - Piedecuesta,1,3.08,1,domingo,6.04,52,2023
2,BC 044 - Piedecuesta,2,3.05,1,domingo,6.08,52,2023
3,BC 044 - Piedecuesta,3,3.01,1,domingo,6.12,52,2023
4,BC 044 - Piedecuesta,4,2.99,1,domingo,6.17,52,2023
...,...,...,...,...,...,...,...,...
1219,BC 90 - Megamall,19,2.01,1,domingo,6.79,52,2023
1220,BC 90 - Megamall,20,1.92,1,domingo,6.83,52,2023
1221,BC 90 - Megamall,21,1.92,1,domingo,6.88,52,2023
1222,BC 90 - Megamall,22,1.90,1,domingo,6.92,52,2023


In [9]:
front_hour = front.groupby(['device_name', 'hour']).mean().round(2).reset_index()

In [17]:
front_hour = front.groupby(['device_name', 'hour']).agg({'value': 'mean', 'day': 'first', 'dow':'first',
       'cont_dow':'first', 'week':'first', 'year':'first' }).reset_index().round(2)

In [21]:
front_hour

,device_name,hour,value,day,dow,cont_dow,week,year
0,BC 044 - Piedecuesta,0,3.08,1,domingo,6.00,52,2023
1,BC 044 - Piedecuesta,1,3.08,1,domingo,6.04,52,2023
2,BC 044 - Piedecuesta,2,3.05,1,domingo,6.08,52,2023
3,BC 044 - Piedecuesta,3,3.01,1,domingo,6.12,52,2023
4,BC 044 - Piedecuesta,4,2.99,1,domingo,6.17,52,2023
...,...,...,...,...,...,...,...,...
1219,BC 90 - Megamall,19,2.01,1,domingo,6.79,52,2023
1220,BC 90 - Megamall,20,1.92,1,domingo,6.83,52,2023
1221,BC 90 - Megamall,21,1.92,1,domingo,6.88,52,2023
1222,BC 90 - Megamall,22,1.90,1,domingo,6.92,52,2023


In [24]:
front_hour = front_hour[['device_name', 'hour', 'value']]

In [25]:
front_hour.columns

Index(['device_name', 'hour', 'value'], dtype='object')

In [26]:
energia = front_hour.pivot(index='device_name', columns='hour', values='value')

In [27]:
energia

hour,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
device_name,,,,,,,,,,,,,,,,,,,,,
BC 044 - Piedecuesta,3.08,3.08,3.05,3.01,2.99,3.01,2.94,8.85,18.63,19.40,...,20.10,20.26,20.09,13.32,3.56,3.18,3.15,3.09,3.08,3.06
BC 061 - Carrera Primera,8.65,8.61,8.73,8.69,8.62,8.52,8.71,12.77,19.52,20.72,...,21.77,21.84,21.18,14.53,8.55,8.46,8.52,8.69,8.67,8.65
BC 185 - Llano Grande Palmira,2.35,2.28,2.27,2.28,2.24,2.24,2.74,7.61,10.84,12.05,...,13.04,13.01,12.78,11.40,4.38,2.51,2.50,2.47,2.38,2.38
BC 197 - Guatapuri,5.69,5.73,5.70,5.69,5.74,5.69,5.61,6.64,12.47,14.49,...,15.44,15.56,15.41,14.89,12.92,10.86,9.26,5.92,5.85,5.79
BC 205 - Villa Colombia,0.48,0.10,0.07,0.08,0.07,0.08,0.08,4.76,10.91,11.05,...,11.84,11.71,11.59,10.08,1.12,0.65,0.53,0.59,0.81,0.50
BC 210 - Banca Colombia Cartagena,1.58,1.54,1.50,1.49,1.50,1.51,1.51,9.10,13.59,13.45,...,10.57,10.40,9.62,7.85,3.51,1.76,1.69,1.64,1.62,1.57
BC 216 - Sabana de Torres,2.19,2.14,2.13,2.11,2.03,1.82,1.81,5.32,9.17,8.95,...,10.44,30.63,9.44,6.56,2.71,2.45,2.32,2.28,2.21,2.23
BC 253 - Puerta del Norte,3.44,3.38,3.35,3.32,3.31,3.27,3.19,3.41,8.10,13.47,...,13.19,13.11,12.90,12.07,8.38,5.45,4.39,3.59,3.45,3.44
BC 291 - Las Palmas,6.88,6.65,6.51,6.37,6.25,6.14,6.09,12.52,20.22,20.85,...,20.19,20.26,19.89,18.86,9.03,7.70,7.52,7.32,7.12,6.98


In [28]:
energia.to_clipboard(index=True, header=True)